# Thématique 7 - Les règles d’association

### Importation des librairies

In [31]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Data pre-processing

In [32]:
# Préparation du fichier de données
filepath = "../data"
filename = "Titanic.csv"
file = os.path.join(filepath, filename)

# Lecture des données
df_original = pd.read_csv(file, delimiter=";")
df = df_original.copy()

# Affichage des indicateurs de positions et de dispersions
df.describe()

,Class,Age,Sex,Survived
count,2201,2201,2201,2201
unique,4,2,2,2
top,crew,adult,male,no
freq,885,2092,1731,1490


Le jeu de données Titanic ressence des informations sur ses passagers, notamment la classe, l'âge, le sexe et s'il a survécu ou non. Le but de cette étude est de connaître les chances de survie d'une personne en fonction de ses éléments en créant des règles d'association.

In [33]:
df.head()

,Class,Age,Sex,Survived
0,first,adult,male,yes
1,first,adult,male,yes
2,first,adult,male,yes
3,first,adult,male,yes
4,first,adult,male,yes


In [34]:
df.tail()

,Class,Age,Sex,Survived
2196,crew,adult,female,yes
2197,crew,adult,female,yes
2198,crew,adult,female,no
2199,crew,adult,female,no
2200,crew,adult,female,no


In [35]:
df.shape

(2201, 4)

On sait qu'il y a 33 pays pour 3 variables à étudier.

In [36]:
df.dtypes

Class       object
Age         object
Sex         object
Survived    object
dtype: object

Dans cette étude, nous avons que des variables qualitatives.

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2201 entries, 0 to 2200
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Class     2201 non-null   object
 1   Age       2201 non-null   object
 2   Sex       2201 non-null   object
 3   Survived  2201 non-null   object
dtypes: object(4)
memory usage: 68.9+ KB


In [38]:
df.columns

Index(['Class', 'Age', 'Sex', 'Survived'], dtype='object')

In [39]:
df.isnull().sum()

Class       0
Age         0
Sex         0
Survived    0
dtype: int64

Le jeu de données n'a pas l'air de présenter de données manquantes.

In [40]:
list_col = df.columns

for col in list_col: 
    print('{} :{} ' . format(col.upper(), df[col].unique()))

CLASS :['first' 'second' 'third' 'crew'] 
AGE :['adult' 'child'] 
SEX :['male' 'female'] 
SURVIVED :['yes' 'no'] 


In [41]:
df

,Class,Age,Sex,Survived
0,first,adult,male,yes
1,first,adult,male,yes
2,first,adult,male,yes
3,first,adult,male,yes
4,first,adult,male,yes
...,...,...,...,...
2196,crew,adult,female,yes
2197,crew,adult,female,yes
2198,crew,adult,female,no
2199,crew,adult,female,no


In [44]:
# Conversion des données en listes de transactions où chaque ligne correspond à une transaction avec ses colonnes comme items
data = df.values.tolist()

data

[['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 'yes'],
 ['first', 'adult', 'male', 

In [43]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Transformation en format transactionnel
te = TransactionEncoder()
te_data = te.fit(data).transform(data)
df_transactions = pd.DataFrame(te_data, columns=te.columns_)

# Extraction des itemsets fréquents avec Apriori
frequent_itemsets = apriori(df_transactions, min_support=0.1, use_colnames=True)

# Génération des règles d'association
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

# Filtrage des règles où l'item de sortie est "Survived=yes"
rules_survivor = rules[rules['consequents'].apply(lambda x: 'yes' in x)]

# Affichage des règles intéressantes
print(rules_survivor)

        antecedents   consequents  antecedent support  consequent support  \
12         (female)         (yes)            0.213539            0.323035   
23  (adult, female)         (yes)            0.193094            0.323035   
25         (female)  (adult, yes)            0.213539            0.297138   

     support  confidence      lift  leverage  conviction  zhangs_metric  
12  0.156293    0.731915  2.265745  0.087312    2.525187       0.710327  
23  0.143571    0.743529  2.301699  0.081195    2.639542       0.700873  
25  0.143571    0.672340  2.262724  0.080121    2.145099       0.709577  


La première règle (female → yes) montre une forte probabilité de survie pour les femmes, avec un lift de 2.27, ce qui signifie que les femmes avaient plus de chances de survivre que les autres passagers. La confiance est également élevée (73.19%).

La deuxième règle (adult, female → yes) est encore plus intéressante car elle suggère que les femmes adultes ont une probabilité encore plus élevée de survivre (74.35%) avec un lift supérieur.

La troisième règle (female → adult, yes) montre que la majorité des femmes qui ont survécu étaient des adultes, mais cette règle a une confiance un peu plus faible (67.23%).

Donc les chances de survies sont bien plus importantes s'il s'agit d'une femme et encore plus si elle est adulte.